# Pororo의 NER Tagger를 이용한 Data Augmentation

### PORORO: Platform Of neuRal mOdels for natuRal language prOcessing [Github link](https://github.com/kakaobrain/pororo)


## 목표: 데이터 중 질문("question") 문장에 NER 태그를 추가하는 data augmentation을 수행합니다.

예시) '해마다 식중독으로 목숨을 잃는 인구는?' -> '해마다 질병 식중독으로 목숨을 잃는 인구는?'




## 🛠 가상환경 세팅

```
conda create -n pororo_env python=3.8
conda activate pororo_env
pip install pororo
pip install datasets==1.5.0  # 중요!!
pip install jupyter
# VSCode 종료 후 재실행
# 우측 상단 select kernel 클릭 후 'pororo_env':conda 선택
# 노트북 실행
conda deactivate # 실험 완료 시
```

In [1]:
from pororo import Pororo
from datasets import load_from_disk
from typing import List, Tuple
import numpy as np

In [2]:
ner = Pororo(task="ner", lang="ko")

In [3]:
# pororo ner task에서 쓰인 전체 태그 목록을 번역하여 매핑함
ENG_TO_KOR = {'PERSON' : '사람', 'LOCATION' : '장소', 'ORGANIZATION' : '조직', 'ARTIFACT' : '인공물',
            'DATE' : '날짜', 'TIME' : '시간', 'CIVILIZATION' : '역할', 'ANIMAL' : '동물',
            'PLANT' : '식물', 'QUANTITY' : '수량', 'STUDY_FIELD' : '분야', 'THEORY' : '이론',
            'EVENT' : '사건', 'MATERIAL' : '물질', 'TERM' : '용어', 
            'OCCUPATION' : '직업', 'COUNTRY' : '국가', 'CITY' : '도시', 'DISEASE' :'질병'}

# 'O': 태그가 없는 경우
# 직접 지정한 부자연스러운 태그 집합 (아래 설명)
IGNORE_TAGS = ('O', 'ARTIFACT', 'DATE', 'TIME', 'CIVILIZATION', 'QUANTITY', 'TERM')

## 증강된 데이터를 눈으로 확인해본 결과, 부자연스러운 태그가 많이 발견되었습니다. 따라서 `'ARTIFACT', 'DATE', 'TIME', 'CIVILIZATION', 'QUANTITY', 'TERM'` 총 6가지 태그는 제외하고 진행했습니다. 부자연스러운 문장의 예시는 아래와 같습니다.

> ARTIFACT: 인공물

'노선이 확대되기 전 인공물 열차의 객차는 몇 개였나?',

'인공물 아다케부네가 제작 된 시대는?',

 '《인공물 가우스전자》의 원래 제목은 무엇이었나?',
 

> DATE: 날짜

 '날짜 고려시대 때 마을의 걱정을 없애기 위해 만든 석불은 무엇으로 만들어졌나요?',

 '국가 스웨덴과 함께 날짜 겨울 전쟁이 끝나길 바란 국가는?',

 '이론 풍수지리사상의 관점에서 보면 날짜 선몽대는 어떤 유형에 속하나요?',


> TIME: 시간

 '사람 게바라가 7월 23일 시간 오후에 간 곳은 어디인가?',

 '비가 내린 뒤 시간 밤에 결혼비행을 하는 개체는 무엇인가?',

 '시간 90분이 종료된 이후 더 주어진 시간동안 몇 골을 성공시켰는가?',


> CIVILIZATION: 역할

 '정치하는 역할 엄마들이 문명 피해자들이 정상적인 일상생활을 할 수 있도록 돕기 위해 진행했던 서비스는?',

 '역할 수령자가 매매계약을 파기하기 위해서 지불해야하는 것은?',

 '사람 코페르니쿠스가 역할 교회법 박사 학위를 받은 날은?',
 
> QUANTITY: 수량

 '사람 카이가 수량 22살에 한 일은?',

 '수량 제 1차 사건 바르바리 전쟁에서 지휘를 맡았던 사람은?',

 '사람 허계임과 그녀의 수량 두 딸 중 가장 먼저 순교한 것은 누구인가?',

> TERM: 용어

 '용어 뇌에 탈피 신호를 보내는 부위는?',

 '완장에 있는 용어 별모양과 비슷한 모양의 빛을 뿜는 것은 누구의 동물 손인가?',
 
 '전기화학적 기울기가 용어 ATP와 무엇을 만들어낼 수 있는가?',

In [4]:
def random_aug(
    word_tag_pairs: List[Tuple[str, str]],
    valid_tag: List[bool],
    max_tag_num: int = 2) -> str:
    '''
    문장에 NER 태그를 random하게 추가해주는 함수.

    Args:
        word_tag_pairs: 입력 문장의 단어와 NER 태그 쌍을 담은 리스트
        valid_tag: 의미 있는 태그를 갖는지에 대한 여부를 나타내는 리스트
        max_tag_num: 출력 문장에 추가할 태그의 최대 개수
    Returns:
        aug_sentence: NER 태그가 단어 앞에 추가된 문장
    """
    '''

    tag_num = sum(valid_tag)  # total num of tags
    
    if tag_num > max_tag_num:
        valid_tag = np.array(valid_tag)
        tag_pos_idx = np.where(valid_tag)

        # leave only max_tag_num positions to True
        set_false_pos = np.random.choice(tag_pos_idx[0], tag_num - max_tag_num, replace=False)
        valid_tag[set_false_pos] = False
    
    aug_sentence = ''
    # create a new sentence with tags inserted
    for item, add_tag in zip(word_tag_pairs, valid_tag):
        if add_tag and item[1] not in IGNORE_TAGS:
            aug_sentence += f'{ENG_TO_KOR[item[1]]} '
        aug_sentence += item[0]      
    return aug_sentence


def add_ner_tag(examples):
    """
    데이터 증강 시 map에서 적용할 함수.
    batch 데이터를 받으면 "question" 문장들을 변형시켜 batch 데이터를 반환함.
    """
    new_questions = []
    for question in examples["question"]:
        ner_question = ner(question)  # list of (word, tag)
        tag_pos = [False if word[1] in IGNORE_TAGS else True for word in ner_question]
        new_questions.append(random_aug(ner_question, tag_pos, max_tag_num=2))
    return {"question": new_questions}

## 데이터 증강에 쓰일 데이터셋을 load합니다.

In [5]:
datasets = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = datasets["train"]

## `datasets.Dataset.map()`을 이용하여 batch 단위로 데이터 증강을 진행합니다.

시간이 꽤 많이 소요됩니다. (-30분) 따라서 우선 sample 30개만으로 합니다.

In [6]:
train_dataset = train_dataset.select(range(30))  # sample

In [7]:
train_dataset_aug = train_dataset.map(
                        add_ner_tag,
                        batched=True,
                        batch_size=8,
                        # num_proc=4  # 2 이상 설정 시, Pororo NER에서 에러 발생
                    )

[Korean Sentence Splitter]: Initializing Pynori...


## `datasets.Dataset.save_to_disk()`로 저장합니다.
`save_to_disk()`로 저장하면 `load_from_disk()`로 쉽게 불러올 수 있습니다.

In [8]:
datasets["train"] = train_dataset_aug
datasets.save_to_disk("/opt/ml/data/ner_only_train_dataset_3")

## `datasets.Dataset.load_from_disk()`로 load까지 성공했는지 확인합니다.

In [9]:
datasets_to_aug = load_from_disk("/opt/ml/data/ner_only_train_dataset_3/")
train_dataset_to_aug = datasets_to_aug["train"]
# type(train_dataset_to_aug)  # datasets.arrow_dataset.Dataset
len(train_dataset_to_aug)  # 30

30